# Sample Maps with shared utils

In [1]:
import branca
import geopandas as gpd
import pandas as pd

import shared_utils 

# Alternatively, can import relevant ones
from shared_utils import altair_utils, map_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0324 17:19:40.183090771    1149 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0324 17:19:42.423742259    1149 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
shared_utils.geography_utils.CA_NAD83Albers

'EPSG:3310'

In [3]:
GEOSPATIAL_DATA = ("https://caltrans-gis.dot.ca.gov/arcgis/rest/services/"
       "CHboundary/District_Tiger_Lines/FeatureServer/0/query?"
       "where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&"
       "inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&"
       "relationParam=&outFields=DISTRICT%2C+Region&returnGeometry=true&"
       "maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&gdbVersion=&"
       "historicMoment=&returnDistinctValues=false&returnIdsOnly=false&"
       "returnCountOnly=false&returnExtentOnly=false&orderByFields=&"
       "groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&"
       "multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&"
       "returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&"
       "returnCentroid=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&"
       "datumTransformation=&f=geojson"
      )

gdf = gpd.read_file(GEOSPATIAL_DATA)

# Create a y-col to map (that's not just District number)
gdf = gdf.assign(
    some_y_col = gdf.index+10
)

gdf

,DISTRICT,Region,geometry,some_y_col
0,1,North,"POLYGON ((-123.51819 42.00031, -123.51794 42.0...",10
1,2,North,"POLYGON ((-122.30683 42.00816, -122.32039 42.0...",11
2,3,North,"POLYGON ((-121.40463 40.14665, -121.40452 40.1...",12
3,4,,"POLYGON ((-122.35190 38.83627, -122.35432 38.8...",13
4,5,Central,"POLYGON ((-122.15057 37.27596, -122.15058 37.2...",14
5,6,Central,"POLYGON ((-119.27249 37.73954, -119.27299 37.7...",15
6,8,,"POLYGON ((-115.66384 35.80924, -115.68169 35.8...",16
7,10,Central,"POLYGON ((-119.88972 38.92320, -119.89712 38.9...",17
8,11,,"POLYGON ((-117.36428 33.50503, -117.43208 33.5...",18
9,12,,"MULTIPOLYGON (((-117.91578 33.94693, -117.9197...",19


In [4]:
print(f"original CRS: {gdf.crs}")
gdf = gdf.to_crs(shared_utils.geography_utils.CA_StatePlane)
print(f"new CRS: {gdf.crs}")

original CRS: epsg:4326
new CRS: EPSG:2229


In [5]:
shared_utils.utils.geoparquet_gcs_export??

Signature: shared_utils.utils.geoparquet_gcs_export(gdf, GCS_FILE_PATH, FILE_NAME)
Source:   
def geoparquet_gcs_export(gdf, GCS_FILE_PATH, FILE_NAME):
    """
    Save geodataframe as parquet locally,
    then move to GCS bucket and delete local file.

    gdf: geopandas.GeoDataFrame
    GCS_FILE_PATH: str. Ex: gs://calitp-analytics-data/data-analyses/my-folder/
    FILE_NAME: str. Filename.
    """
    gdf.to_parquet(f"./{FILE_NAME}.parquet")
    fs.put(f"./{FILE_NAME}.parquet", f"{GCS_FILE_PATH}{FILE_NAME}.parquet")
    os.remove(f"./{FILE_NAME}.parquet")
File:      ~/data-analyses/_shared_utils/shared_utils/utils.py
Type:      function


In [6]:
GCS_FILE_PATH = "gs://calitp-analytics-data/"
FILE_NAME = "test_geoparquet"
shared_utils.utils.geoparquet_gcs_export(gdf, GCS_FILE_PATH, FILE_NAME)

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


In [7]:
# Set your own step colormap and specify colors and cut-offs
colorscale = branca.colormap.StepColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

print("Step colorscale, define colors, cut-offs")
display(colorscale)

# Set your own linear colormap and specify colors and cut-offs
colorscale = branca.colormap.LinearColormap(
                colors=["gray", "green", "navy"], 
                index=[2_000, 4_000, 8_000],
                vmin=0, vmax=15_000,
)

print("Linear colorscale, define colors, cut-offs")
display(colorscale)

# Use preset step colormap
colorscale = branca.colormap.step.Accent_03

print("Preset step colorscale")
display(colorscale)

# Use preset linear colormap
colorscale = branca.colormap.linear.Blues_06

print("Preset linear colorscale")
display(colorscale)

colorscale = (branca.colormap.step.Accent_07
              .scale(vmin=gdf.DISTRICT.min(), 
                     vmax=gdf.DISTRICT.max())
             )
print("Preset step colorscale, define min/max")
display(colorscale)

Step colorscale, define colors, cut-offs


Linear colorscale, define colors, cut-offs


Preset step colorscale


Preset linear colorscale


Preset step colorscale, define min/max


In [8]:
popup_dict = {
    "DISTRICT": "Caltrans District",
    "Region": "Caltrans Region",
    "some_y_col": "Something Measured"
}

colorscale = (branca.colormap.step.Accent_07
              .scale(vmin=gdf.DISTRICT.min(), 
                     vmax=gdf.DISTRICT.max())
             )

m = shared_utils.map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "DISTRICT",
    popup_dict = popup_dict,
    tooltip_dict = popup_dict,
    colorscale = colorscale,
    fig_width = 500, fig_height = 800,
    zoom = shared_utils.map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid = shared_utils.map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="CA Map"
)

#m

In [9]:
# Add additional kwargs to folium map
colorscale = branca.colormap.StepColormap(
                colors=["gray", "green", "navy"], 
                vmin=gdf.DISTRICT.min(), vmax=gdf.DISTRICT.max(),
)

m = shared_utils.map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "DISTRICT",
    popup_dict = popup_dict,
    tooltip_dict = popup_dict,
    colorscale = colorscale,
    fig_width = 500, fig_height = 800,
    zoom = shared_utils.map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid = shared_utils.map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title="CA Map",
    # Additional kwargs
    # folium.Map()
    zoom_control=False,
    # folium.features.GeoJson
    zoom_on_click = True,
)

#m

In [10]:
gdf = gdf.to_crs(shared_utils.geography_utils.WGS84)

choropleth_dict = {
    "layer_name": "some_y_col",
    "MIN_VALUE": gdf.some_y_col.min(),
    "MAX_VALUE": gdf.some_y_col.max(),
    "plot_col_name": "some_y_col",
    "fig_width": '100%',
    "fig_height": '100%',
    "fig_min_width_px": '400px',
    "fig_min_height_px": '600px',
}

colorscale = branca.colormap.linear.Greens_09

print("Preset linear colorscale")
display(colorscale)

f = shared_utils.map_utils.make_ipyleaflet_choropleth_map(
    gdf,
    geometry_col = "DISTRICT", 
    plot_col = "some_y_col",
    choropleth_dict = choropleth_dict,
    colorscale = colorscale
)

f

Preset linear colorscale


Map(center=[35.8, -119.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…